In [ ]:
import pandas as pd
import glob
import os
import re

In [ ]:
def create_dataframe_in_folder_with_date_field(folder_path, schema, date_regular_expression):

    """
    Goal: Create a dataframe that reads all CSV files from a file path and add a column "file_extraction_date" based on the CSV name

    folder_path: Folder path where the CSV from amazon data are being stored -> str
    schema: Schema of the dataframe -> dict
    date_regular_expression: Regular expression to extract the file date from the file name -> regex
    
    """

    # Folder containing your CSV files
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

    df_list = []

    for file in csv_files:
        # Get just the filename
        filename = os.path.basename(file)

        # Extract YYYY-MM from the filename based on a regular expression
        match = re.search(date_regular_expression, filename)
        file_date = match.group(0) if match else 'unknown'

        try:
            df = pd.read_csv(file, dtype=schema)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue

        # Insert the date as the first column
        df.insert(0, 'file_extraction_date', file_date)

        df_list.append(df)

    # Combine all dataframes
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        combined_df.head(5)
    else:
        print("No data loaded.")

    return combined_df

In [ ]:
def create_dataframe(file_path, schema):
    """
    Goal: Create a dataframe from a single CSV file

    file_path: File path with file name csv extension -> str
    schema: Schema of the dataframe -> dict
    
    """

    # Read the CSV into a DataFrame
    df = pd.read_csv(file_path, dtype=schema)

    # Display the first 5 rows
    df.head(5)

    return df

In [ ]:
def write_to_db(df, database_name, table_name):

    """
    Goal: Write a pandas dataframe to SQLite database 

    df: Dataframe to be written -> df
    database_name: Database name that it is going to be created the table -> str
    table_name: Table name that is going to be created on database_name -> str
    
    """

    try:
        conn = sqlite3.connect(database_name)

        # Save to SQLite
        df.to_sql(table_name, conn, if_exists='replace', index=False)

        # Verify
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM amazon_data")
        for row in cursor.fetchall():
            print(row)

        conn.close()

        print("Table creation Successful!")
    except Exception:
        print("Connection was not successful. Please, review the parameters.")

    return ""

In [ ]:
def clean_title(title):

    if pd.isna(title):  # If title is null/NA, return None
        return None

    title = title.strip()

    # Rule 1: If colon exists, truncate at the colon
    if ':' in title:
        title = title.split(':')[0].strip()

    # Rule 2: If title starts with [, (, or { and contains multiple bracketed groups
    if re.match(r'^[\[\(\{]', title):
        groups = re.findall(r'[\[\(\{]([^()\[\]{}]+)[\]\)\}]', title)
        if groups:
            return groups[0].strip()

    # Rule 3: Otherwise, remove everything after the first (
    title = re.split(r'\s*\(', title)[0].strip()

    # Rule 4: Remove remaining brackets or braces
    title = re.sub(r'^[\[\(\{](.*?)[\]\)\}]$', r'\1', title).strip()

    return title